In [8]:
import pandas as pd

### Unindo os datasets
Unir as notas de bechdel com elenco e equipe com o merge:
- O primeiro parâmetro passado é dataframe da "esquerda" e o segundo, da direita. Os dois próximos parâmetros (`left_on` e `right_on`) dizem quais colunas serão consideradas pra fazer o merge. O `how='inner'` calcula a interseção entre essas duas colunas.

Primeiro é feito o merge do links_df com o bechdel_df, pois bechdel_df não possui o tmdbid diretamente, mas os outros sim.
obs.: o 'id' no `credits.csv` e no `movies_metadata.csv` é o tmdbid.

`reset_index(drop='True')` = reseta os indexes sem criar uma nova coluna para eles.

In [95]:
bechdel_df = pd.read_csv('datasets/Bechdel_detailed.csv')
links_df = pd.read_csv('datasets/movie-dataset/links.csv') # contém o id do imdb e tmdb de cada filme.
credits_df = pd.read_csv('datasets/movie-dataset/credits.csv')
imdb_df = pd.read_csv('datasets/movie-dataset/movies_metadata.csv')

# remove colunas não usadas.
bechdel_df = bechdel_df.drop(['Unnamed: 0', 'submitterid', 'date', 'visible', 'id'], axis=1)

# renomeia rating para não coincidir com o dos outros datasets.
bechdel_df = bechdel_df.rename(columns={'rating': 'bt_score'})

# junta os 3 dataframes e descarta colunas duplicadas.
bechdel_df = pd.merge(bechdel_df, links_df, left_on='imdbid', right_on='imdbId', how='inner').drop('imdbId', axis=1)
bechdel_df = pd.merge(bechdel_df, credits_df, left_on='tmdbId', right_on='id', how='inner').drop(['id', 'movieId'], axis=1)

# descarta itens sem cast e/ou crew. Reseta os index.
bechdel_df = bechdel_df[
	(bechdel_df.cast != '[]') & (bechdel_df.crew != '[]')
].reset_index(drop='True')

# UNINDO COM O MOVIES_METADATA.CSV

# transforma o dtype de todos os elementos da coluna 'id' em float pra poder fazer o merge. Alguns possuem strings que são então convertidas em NaN.
imdb_df['id'] = pd.to_numeric(imdb_df['id'], errors='coerce')

# remove colunas não usadas.
imdb_df = imdb_df.drop(['belongs_to_collection', 'homepage', 'original_title', 'overview', 'poster_path', 'release_date', 'spoken_languages', 'status', 'tagline', 'video', 'title'], axis=1)

# faz o df final.
bechdel_df = pd.merge(bechdel_df, imdb_df, left_on='tmdbId', right_on='id', how='inner').drop('id', axis=1)

bechdel_df.head()

/var/folders/x5/46svw_qj6hs7h_f0d9_k70rc0000gn/T/ipykernel_8001/2704256597.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  imdb_df = pd.read_csv('datasets/movie-dataset/movies_metadata.csv')


,title,year,bt_score,dubious,imdbid,tmdbId,cast,crew,adult,budget,genres,imdb_id,original_language,popularity,production_companies,production_countries,revenue,runtime,vote_average,vote_count
0,Sallie Gardner at a Gallop,1878.0,0.0,0.0,2221420.0,194079.0,"[{'cast_id': 5, 'character': 'Herself - Sallie...","[{'credit_id': '52fe4ce29251416c91103327', 'de...",False,0,"[{'id': 99, 'name': 'Documentary'}]",tt2221420,en,0.327841,"[{'name': 'Palo Alto Stock Farm', 'id': 84748}]","[{'iso_3166_1': 'US', 'name': 'United States o...",0.0,1.0,6.2,25.0
1,Accordion Player,1888.0,0.0,0.0,1758563.0,96882.0,"[{'cast_id': 1, 'character': 'Himself', 'credi...","[{'credit_id': '52fe49c39251416c750d2a8b', 'de...",False,0,"[{'id': 99, 'name': 'Documentary'}]",tt1758563,xx,0.212768,"[{'name': 'Whitley Partners', 'id': 48026}]","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'}]",0.0,1.0,4.4,18.0
2,"Monkeyshines, No. 1",1890.0,0.0,0.0,361921.0,32571.0,"[{'cast_id': 10, 'character': 'Himself', 'cred...","[{'credit_id': '52fe44d99251416c9101ef9f', 'de...",False,0,"[{'id': 35, 'name': 'Comedy'}]",tt0361921,en,1.163672,"[{'name': 'Edison Manufacturing Company', 'id'...","[{'iso_3166_1': 'US', 'name': 'United States o...",0.0,1.0,4.3,15.0
3,"Monkeyshines, No. 2",1890.0,0.0,0.0,416046.0,33315.0,"[{'cast_id': 1, 'character': 'Himself', 'credi...","[{'credit_id': '52fe450c9251416c91025b0b', 'de...",False,0,[],tt0416046,xx,0.399258,"[{'name': 'Edison Manufacturing Company', 'id'...","[{'iso_3166_1': 'US', 'name': 'United States o...",0.0,1.0,4.5,12.0
4,"Monkeyshines, No. 3",1890.0,0.0,0.0,416047.0,33317.0,"[{'cast_id': 1, 'character': 'Himself', 'credi...","[{'credit_id': '52fe450c9251416c91025b2f', 'de...",False,0,[],tt0416047,xx,0.276161,"[{'name': 'Edison Manufacturing Company', 'id'...","[{'iso_3166_1': 'US', 'name': 'United States o...",0.0,1.0,3.3,5.0
